<a href="https://colab.research.google.com/github/saadz-khan/federated-learning/blob/master/PySyft_Full_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing PySyft

In [ ]:
!pip install syft==0.2.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 433 kB 5.7 MB/s 
     |████████████████████████████████| 9.0 MB 28.6 MB/s 
     |████████████████████████████████| 79 kB 1.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 484 kB 45.3 MB/s 
     |████████████████████████████████| 57 kB 4.6 MB/s 
     |████████████████████████████████| 449 kB 45.0 MB/s 
     |████████████████████████████████| 66 kB 4.3 MB/s 
     |████████████████████████████████| 2.0 MB 40.7 MB/s 
     |████████████████████████████████| 126 kB 47.8 MB/s 
     |████████████████████████████████| 1.8 MB 37.1 MB/s 
     |████████████████████████████████| 73 kB 2.0 MB/s 
     |████████████████████████████████| 26.1 MB 79.7 MB/s 
     |████████████████████████████████| 753.4 MB 6.6 kB/s 
     |████████████████████████████████| 4.0 MB 27.6 MB/s 
     |████████████████████████████████

## Importing Dependencies

In [ ]:
import sys
import torch

import syft as sy

import torchvision
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

## Hooking PySyft to PyTorch

In [ ]:
hook = sy.TorchHook(torch)

## Creating Workers

In [ ]:
jake = sy.VirtualWorker(hook, id="jake")
john = sy.VirtualWorker(hook, id="john")
jason = sy.VirtualWorker(hook, id="jason")

## Setting Up DataSet

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, )),
])

train_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=True, download=True, transform=transform)
test_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=False, download=True, transform=transform)

""" Distributing the Local Dataset among Workers to Simulate Federated Learning """
federated_train_loader = sy.FederatedDataLoader(
    train_set.federate((jake, john, jason)), batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=64, shuffle=True)

0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw
Processing...
Done!


## Building an ML Model

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Training The Model

In [ ]:
for epoch in range(0, 5):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        # send the model to the client device where the data is present
        model.send(data.location)
        # training the model
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # get back the improved model
        model.get()
        if batch_idx % 100 == 0:
            # get back the loss
            loss = loss.get()
            print('Epoch: {:2d} [{:5d}/{:5d} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                batch_idx * 64,
                len(federated_train_loader) * 64,
                100. * batch_idx / len(federated_train_loader),
                loss.item()))

Epoch:  1 [    0/60032 (  0%)]	Loss: 2.330317


Exception ignored in: <function ObjectPointer.__del__ at 0x7f630d45f0e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/syft/generic/pointers/object_pointer.py", line 345, in __del__
  File "/usr/local/lib/python3.7/dist-packages/syft/workers/base.py", line 480, in garbage
  File "/usr/local/lib/python3.7/dist-packages/syft/workers/base.py", line 316, in send_msg
  File "/usr/local/lib/python3.7/dist-packages/syft/workers/virtual.py", line 12, in _send_msg
  File "/usr/local/lib/python3.7/dist-packages/syft/workers/virtual.py", line 22, in _recv_msg
  File "/usr/local/lib/python3.7/dist-packages/syft/workers/base.py", line 356, in recv_msg
  File "/usr/local/lib/python3.7/dist-packages/syft/generic/abstract/message_handler.py", line 20, in handle
  File "/usr/local/lib/python3.7/dist-packages/syft/workers/message_handler.py", line 211, in handle_force_delete_object_msg
  File "/usr/local/lib/python3.7/dist-packages/syft/generic/object_storage.py", lin

Epoch:  1 [ 6400/60032 ( 11%)]	Loss: 1.418613
Epoch:  1 [12800/60032 ( 21%)]	Loss: 0.666170
Epoch:  1 [19200/60032 ( 32%)]	Loss: 0.692633
Epoch:  1 [25600/60032 ( 43%)]	Loss: 0.622887
Epoch:  1 [32000/60032 ( 53%)]	Loss: 0.597961
Epoch:  1 [38400/60032 ( 64%)]	Loss: 0.380533
Epoch:  1 [44800/60032 ( 75%)]	Loss: 0.295889
Epoch:  1 [51200/60032 ( 85%)]	Loss: 0.341914
Epoch:  1 [57600/60032 ( 96%)]	Loss: 0.336795
Epoch:  2 [    0/60032 (  0%)]	Loss: 0.314997
Epoch:  2 [ 6400/60032 ( 11%)]	Loss: 0.238472
Epoch:  2 [12800/60032 ( 21%)]	Loss: 0.372945
Epoch:  2 [19200/60032 ( 32%)]	Loss: 0.477317
Epoch:  2 [25600/60032 ( 43%)]	Loss: 0.354022
Epoch:  2 [32000/60032 ( 53%)]	Loss: 0.261799
Epoch:  2 [38400/60032 ( 64%)]	Loss: 0.433237
Epoch:  2 [44800/60032 ( 75%)]	Loss: 0.244670
Epoch:  2 [51200/60032 ( 85%)]	Loss: 0.284619
Epoch:  2 [57600/60032 ( 96%)]	Loss: 0.420940
Epoch:  3 [    0/60032 (  0%)]	Loss: 0.301005
Epoch:  3 [ 6400/60032 ( 11%)]	Loss: 0.287614
Epoch:  3 [12800/60032 ( 21%)]	Los

## Testing the Model

In [ ]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(
            output, target, reduction='sum').item()
        # get the index of the max log-probability
        pred = output.argmax(1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss,
    correct,
    len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.2473, Accuracy: 9289/10000 (93%)



# Attributed Workers

In [ ]:
class FederatedWorker(sy.VirtualWorker):
  def __init__(self,
               is_idle: bool, is_charging: bool, has_free_wifi: bool, 
               hook, id = 0,
               data = None,
               is_client_worker: bool = False, 
               log_msgs: bool = False, verbose: bool = False, 
               auto_add: bool = True,
               message_pending_time=0):
    super().__init__(hook=hook, id=id, data=data, is_client_worker=is_client_worker, log_msgs=log_msgs, verbose=verbose, auto_add=auto_add, message_pending_time=message_pending_time)
    self.is_idle= is_idle
    self.is_charging=is_charging
    self.has_free_wifi=has_free_wifi

New Attributed Workers

In [ ]:
ash11 = FederatedWorker(is_idle=True, is_charging=True, has_free_wifi=True, hook=hook, id="ash11")
aaron11 = FederatedWorker(is_idle=True, is_charging=False, has_free_wifi=True, hook=hook, id="aaron11")
amy11 = FederatedWorker(is_idle=True, is_charging=True, has_free_wifi=True, hook=hook, id="amy11")

attworkers = {ash11.id:ash11, aaron11.id:aaron11, amy11.id:amy11} 

Distributing Dataset

In [ ]:
federated_dataset = train_set.federate(tuple(attworkers.values()))

In [ ]:
federated_dataset.workers

['ash11', 'aaron11', 'amy11']

In [ ]:
for worker in federated_dataset.workers:
  if not (attworkers[worker].is_idle and attworkers[worker].is_charging and attworkers[worker].has_free_wifi):
    federated_dataset.workers.remove(worker)
    federated_dataset.datasets.pop(worker)



In [ ]:
print(federated_dataset.workers)
print(federated_dataset.datasets)

['ash11', 'amy11']
{'ash11': BaseDataset
	Data: (Wrapper)>[PointerTensor | me:41624335913 -> ash11:84224326496]
	targets: (Wrapper)>[PointerTensor | me:86406716580 -> ash11:16822856969], 'amy11': BaseDataset
	Data: (Wrapper)>[PointerTensor | me:62544220224 -> amy11:83142273018]
	targets: (Wrapper)>[PointerTensor | me:72458872311 -> amy11:85665778246]}


In [ ]:
federated_train_loader = sy.FederatedDataLoader(
    federated_dataset, batch_size=64, shuffle=True)

### Training

In [ ]:
for epoch in range(0, 5):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        # send the model to the client device where the data is present
        model.send(data.location)
        # training the model
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # get back the improved model
        model.get()
        if batch_idx % 100 == 0:
            # get back the loss
            loss = loss.get()
            print('Epoch: {:2d} [{:5d}/{:5d} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                batch_idx * 64,
                len(federated_train_loader) * 64,
                100. * batch_idx / len(federated_train_loader),
                loss.item()))

Epoch:  1 [    0/40000 (  0%)]	Loss: 0.186022
Epoch:  1 [ 6400/40000 ( 16%)]	Loss: 0.181398
Epoch:  1 [12800/40000 ( 32%)]	Loss: 0.192152
Epoch:  1 [19200/40000 ( 48%)]	Loss: 0.066450
Epoch:  1 [25600/40000 ( 64%)]	Loss: 0.286380
Epoch:  1 [32000/40000 ( 80%)]	Loss: 0.172224
Epoch:  1 [38400/40000 ( 96%)]	Loss: 0.297005
Epoch:  2 [    0/40000 (  0%)]	Loss: 0.185656
Epoch:  2 [ 6400/40000 ( 16%)]	Loss: 0.258444
Epoch:  2 [12800/40000 ( 32%)]	Loss: 0.257863
Epoch:  2 [19200/40000 ( 48%)]	Loss: 0.290016
Epoch:  2 [25600/40000 ( 64%)]	Loss: 0.272216
Epoch:  2 [32000/40000 ( 80%)]	Loss: 0.165734
Epoch:  2 [38400/40000 ( 96%)]	Loss: 0.188154
Epoch:  3 [    0/40000 (  0%)]	Loss: 0.291519
Epoch:  3 [ 6400/40000 ( 16%)]	Loss: 0.102859
Epoch:  3 [12800/40000 ( 32%)]	Loss: 0.256702
Epoch:  3 [19200/40000 ( 48%)]	Loss: 0.375760
Epoch:  3 [25600/40000 ( 64%)]	Loss: 0.205670
Epoch:  3 [32000/40000 ( 80%)]	Loss: 0.348682
Epoch:  3 [38400/40000 ( 96%)]	Loss: 0.219919
Epoch:  4 [    0/40000 (  0%)]	Los

### Testing

In [ ]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(
            output, target, reduction='sum').item()
        # get the index of the max log-probability
        pred = output.argmax(1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss,
    correct,
    len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))